In [1]:
from hana_ml.dataframe import ConnectionContext
from data_load_utils import DataSets, Settings
url, port, user, pwd = Settings.load_config("../../config/e2edata.ini")
connection_context = ConnectionContext(url, port, user, pwd)
connection_context.get_current_schema()

import pandas as pd
import numpy as np
from hana_ml.dataframe import create_dataframe_from_pandas
import matplotlib.pyplot as plt

n_samples = 100
df = pd.DataFrame(np.random.randn(n_samples,2))
df.columns = ['X1', 'X2']
df_clustering = create_dataframe_from_pandas(connection_context=connection_context, pandas_df=df, table_name='RAM_TBL', force=True, replace=True)
df_clustering = df_clustering.add_id('ID')

n_samples = 10
df2 = pd.DataFrame(np.random.randn(n_samples,2))
df2.columns = ['X1', 'X2']
df_predict = create_dataframe_from_pandas(connection_context=connection_context, pandas_df=df2, table_name='RAM_PREDICT_TBL', force=True, replace=True)
df_predict = df_predict.add_id('ID')

In [1]:
from hana_ml.dataframe import ConnectionContext
from data_load_utils import DataSets, Settings
url, port, user, pwd = Settings.load_config("../../config/e2edata.ini")
connection_context = ConnectionContext(url, port, user, pwd)
connection_context.get_current_schema()

import unittest
from hana_ml.algorithms.pal.unified_clustering import UnifiedClustering
from hana_ml.model_storage import ModelStorage
import pandas as pd
from pandas.testing import assert_frame_equal, assert_series_equal
import numpy as np
data_ahc=connection_context.table('DATA_UNICU_AHC_TBL')

ahc_params = dict(
         n_clusters=4,
         affinity='Gower',
         linkage='weighted average',
         thread_ratio=None,
         distance_dimension=3,
         normalization=0,
         category_weights=0.1)
uahc = UnifiedClustering(func = 'AgglomerateHierarchicalClustering', **ahc_params)
uahc.fit(data_ahc, key='POINT', categorical_variable=['X3'])

assert_series_equal(uahc.labels_.head(6).collect()['CLUSTER_ID'],
                   pd.Series(data=[1, 1, 1, 1, 1, 2], name = 'CLUSTER_ID'))

uahc.predict(data_ahc, key='POINT')


[('FUNCTION', None, None, 'AHC'), ('N_CLUSTERS', 4, None, None), ('AFFINITY', 10, None, None), ('LINKAGE', 4, None, None), ('THREAD_RATIO', None, None, None), ('DISTANCE_DIMENSION', None, 3, None), ('NORMALIZATION', 0, None, None), ('CATEGORY_WEIGHTS', None, 0.1, None), ('CATEGORICAL_VARIABLE', None, None, 'X3')]


ERROR:hana_ml.algorithms.pal.unified_clustering:AgglomerateHierarchicalClustering does not provide predict function!


ValueError: AgglomerateHierarchicalClustering does not provide predict function!

In [3]:
from hana_ml.dataframe import ConnectionContext
from data_load_utils import DataSets, Settings
url, port, user, pwd = Settings.load_config("../../config/e2edata.ini")
connection_context = ConnectionContext(url, port, user, pwd)
connection_context.get_current_schema()

import unittest
from hana_ml.algorithms.pal.unified_clustering import UnifiedClustering
from hana_ml.model_storage import ModelStorage
import pandas as pd
from pandas.testing import assert_frame_equal, assert_series_equal
import numpy as np
data_dbscan=connection_context.table('DATA_UNICU_DBSCAN_TBL')
data_dbscan_predict=connection_context.table('DATA_UNICU_DBSCAN_PREDICT_TBL')

dbscan_params = dict(metric='manhattan', thread_ratio=0.2)
udbscan = UnifiedClustering(func = 'DBSCAN', **dbscan_params)
udbscan.fit(data_dbscan, key='ID', features=['V1', 'V2', 'V3'])

assert_series_equal(udbscan.labels_.head(15).collect()['CLUSTER_ID'],
                    pd.Series(data=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], name = 'CLUSTER_ID'))
               
result = udbscan.predict(data_dbscan_predict, key='ID' )
expected_result = pd.DataFrame(data={'ID':[1, 2, 3, 4, 5], 'CLUSTER_ID':[0, 0, -1, 1, -1], 'DISTANCE':[0, 2.21, 3, 0.05000000000000071, 3.59]})
assert_frame_equal(result.collect().set_index('ID'),
                   expected_result.set_index('ID'), check_like=True)   

[('FUNCTION', None, None, 'DBSCAN'), ('METRIC', 1, None, None), ('THREAD_RATIO', None, 0.2, None), ('AUTO_PARAM', None, None, 'true')]


In [2]:
from hana_ml.dataframe import ConnectionContext
from data_load_utils import DataSets, Settings
url, port, user, pwd = Settings.load_config("../../config/e2edata.ini")
connection_context = ConnectionContext(url, port, user, pwd)
connection_context.get_current_schema()

import unittest
from hana_ml.algorithms.pal.unified_clustering import UnifiedClustering
from hana_ml.model_storage import ModelStorage
import pandas as pd
from pandas.testing import assert_frame_equal, assert_series_equal
import numpy as np
data_gmm=connection_context.table('DATA_UNICU_GMM_TBL')
data_gmm_predict=connection_context.table('DATA_UNICU_GMM_PREDICT_TBL')

gmm_params = dict(init_param='farthest_first_traversal',
                              n_components=2, covariance_type='full',
                              shared_covariance=False, max_iter=500,
                              error_tol=0.001, thread_ratio=0.5, random_seed=1)
ugmm = UnifiedClustering(func = 'gaussianmixture', **gmm_params)
ugmm.fit(data_gmm, key='ID')

print(ugmm.model_.collect())
expected_result = pd.DataFrame(data={
    'ID':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'CLUSTER_ID':[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
    'DISTANCE' : [None,None,None,None,None,None,None,None,None,None,None,None,None,None,None],
    'SLIGHT_SILHOUETTE' : [None,None,None,None,None,None,None,None,None,None,None,None,None,None,None]})
assert_series_equal(ugmm.labels_.head(15).collect()['CLUSTER_ID'],
                    pd.Series(data=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], name = 'CLUSTER_ID'))
               
result = ugmm.predict(data_gmm_predict, key='ID' )
print(result.collect())
expected_result = pd.DataFrame(data={'ID':[99, 100, 101, 102, 103], 
                                     'CLUSTER_ID':[0, 0, 0, 0, 0], 
                                     'DISTANCE':[1.0, 1.0, 1.0, 0.0, 0.0]})
assert_frame_equal(result.collect().head(5).set_index('ID'),
                   expected_result.set_index('ID'), check_like=True)   

[('FUNCTION', None, None, 'GMM'), ('INIT_MODE', 0, None, None), ('COVARIANCE_TYPE', 0, None, None), ('SHARED_COVARIANCE', False, None, None), ('MAX_ITER', 500, None, None), ('ERROR_TOL', None, 0.001, None), ('THREAD_RATIO', None, 0.5, None), ('SEED', 1, None, None), ('INITIALIZE_PARAMETER', None, None, '2')]
   ROW_INDEX  PART_INDEX                                      MODEL_CONTENT
0          0          -1  {"Algorithm":"GMM","Metadata":{"DataProcessor"...
1          1           0  {"GaussModel":{"covariance":[0.000148222222222...
2          2           1  {"GaussModel":{"covariance":[0.000148222222222...
   ROW_INDEX  PART_INDEX                                      MODEL_CONTENT
0          0          -1  {"Algorithm":"GMM","Metadata":{"DataProcessor"...
1          1           0  {"GaussModel":{"covariance":[0.000148222222222...
2          2           1  {"GaussModel":{"covariance":[0.000148222222222...
     ID  CLUSTER_ID  DISTANCE
0    99           0       1.0
1   100           0   